In [1]:
import json
import pandas as pd
import xlrd
import os

# Salto

In [2]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 124
La longitud de Categoría es: 124
La longitud de Provincia es: 124
La longitud de Localidad es: 124
La longitud de Disciplina es: 124
La longitud de Federación es: 124
La longitud de Resultados es: 124
La longitud de País es: 124
La longitud de Inicio es: 124
La longitud de Final es: 124
La longitud de Ambito es: 124


In [3]:
directorio = "../data/data_salto/resultados_2025"

concursos = []
tipos_pruebas = []
# Obtener la lista de todos los archivos en el directorio
archivos = os.listdir(directorio)

# Filtrar solo los archivos que parecen ser de Excel (puedes refinar esto más si es necesario)
archivos_excel = [f for f in archivos if f.endswith(('.xlsx', '.xls'))]

if archivos_excel:
    # Ordenar la lista de archivos para asegurar que el "primero" sea consistente
    archivos_excel.sort()

    for nombre_excel in archivos_excel:
        ruta_excel = os.path.join(directorio, nombre_excel)
        print(f"\n--- Leyendo archivo: {nombre_excel} ---")
        try:
            # Leer el archivo Excel actual
            df = pd.read_excel(ruta_excel)
            concursos.append(df.iloc[1,1]) # Mostrar las primeras filas de cada DataFrame
            tipos_pruebas.append(df.iloc[3,1])
        except FileNotFoundError:
            print(f"Error: No se encontró el archivo: {ruta_excel}")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo {nombre_excel}: {e}")
else:
        print("No se encontraron archivos Excel en el directorio.")            


--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (1).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (10).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (100).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (11).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (12).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (13).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (14).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (15).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (16).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (17).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (18).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (19).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (2).xls ---

--- Leyendo archivo: RFHE_SALTO_Resultados de la Prueba (20).xls ---

--- Leyendo archivo:

In [4]:
# comprobamos que el numero de excels es correcto
print(len(concursos))
print(len(tipos_pruebas))

1518
1518


In [5]:
# obtenemos los valores unicos de la pruebas
valores_unicos = list(set(tipos_pruebas))
print(len(valores_unicos))
valores_unicos


170


['1,20 - Dos fases Especial (Art. 274.2) (274.2)',
 '1,45 (GP) - A.c.c. y desempate al cronómetro (238.2.2)',
 '1,30 - Caza (239)',
 'Ch - 1,25 m. - 2 fases Asc / Acc (274.1.5.2)',
 '1,10 - Dos Fases (274.1.5.3)',
 '1,10 - A con cronómetro (238.2.1)',
 'YR - 1,40 m. - A c.c. (238.2.1)',
 '1,40 (GP) - A.c.c. y desempate (238.2.2)',
 '1,30 - Accumulator (269.3.2)',
 '6 años - 1,25 - A c.c. (238.2.1)',
 '7 años - 1,35 - A.c.c. y desempate (238.2.2)',
 '6 años - 1,20 - A c.c. (238.2.1)',
 '1,40 (M) - A c.c. (238.2.1)',
 'YR - 1,45 m. - A c.c. (238.2.1)',
 '1,40 gp - Dos Mangas sin desempate (273.3.3.1)',
 '1.15 - Acc (238.2.1)',
 '1,30 - A c.c. (238.2.1)',
 '1,45 - A c.c. (238.2.1)',
 '8 años - 1,40 - A.c.c. (238.2.1)',
 '1,20 - Dos Fases Especial (274.2)',
 'J - CN - C.N. (264)',
 '1,50 (G) - A con cronometro sin desempate (238.2.1)',
 '5 años - 1,10 - A c.c. (238.2.1)',
 '1,30 - Dificultades Progresivas (269.3.2)',
 '1,35 GP - Dos Mangas (A c/c - A c/c) (273.3.3.1)',
 '7 años - 1,40 - A 

In [6]:
# como queremos quedarnos con la parte general de las pruebas, lo sacamos de cada tipo de prueba
tipos_pruebas_bueno = []
for elemento in valores_unicos:
    elemento_bueno = elemento.split("-")
    if len(elemento_bueno) == 3:
        tipos_pruebas_bueno.append(elemento_bueno[2].lstrip())
    elif len(elemento_bueno) == 2:
        tipos_pruebas_bueno.append(elemento_bueno[1].lstrip())

tipos_pruebas_bueno

['Dos fases Especial (Art. 274.2) (274.2)',
 'A.c.c. y desempate al cronómetro (238.2.2)',
 'Caza (239)',
 '2 fases Asc / Acc (274.1.5.2)',
 'Dos Fases (274.1.5.3)',
 'A con cronómetro (238.2.1)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'Accumulator (269.3.2)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'Dos Mangas sin desempate (273.3.3.1)',
 'Acc (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 'A.c.c. (238.2.1)',
 'Dos Fases Especial (274.2)',
 'C.N. (264)',
 'A con cronometro sin desempate (238.2.1)',
 'A c.c. (238.2.1)',
 'Dificultades Progresivas (269.3.2)',
 'A c/c) (273.3.3.1)',
 'A c.c. (238.2.1)',
 'A.c.c. y desempate (238.2.2)',
 'Acc (238.2.1)',
 'A c.c. (238.2.1)',
 'A c.c. (238.2.1)',
 '2 fases Asc / Acc (274.1.5.2)',
 'A.c.c. y desempate (238.2.2)',
 'Dos Fases Especial (274.2)',
 'Dos Fases Especial (274.2)',
 'Dos Fases (274.1.5.3)',
 'A.c.c. y desempate (238.2.2)',
 'A.c.c.

In [7]:
# volvemos a obtener los valores unicos de los tipos de pruebas
pruebas_unicas = list(set(tipos_pruebas_bueno))
pruebas_unicas

['A.c.c. (238.2.1)',
 'A.c.c. y desempate al cronómetro (238.2.2)',
 'A.c.c.) sin desempate (273.3.3.1)',
 'Dos Fases (274.1.5.3)',
 'A c/c y un desempate (238.2.2)',
 'Acc con desempate (238.2.2)',
 'A.c.c y desempate (238.2.2)',
 'Accumulator (269.3.2)',
 'Acc (238.2.1)',
 'A.c.c (238.2.1)',
 'Dificultades Progresivas con Jocker (269.3.2)',
 'Dificultades Progresivas (269.3.2)',
 'A.s.c. y desempate al cronómetro (238.1.2)',
 'Velocidad y Manejabilidad (263)',
 'Dos fases Especial (Art. 274.2) (274.2)',
 'DOS FASES (274.1.5.3)',
 'Dos Fases Especial (274.2)',
 'A.c.c. y desempate (238.2.2)',
 'Acc y desempate (238.2.1)',
 'C (239)',
 'Dos Mangas sin desempate (273.3.3.1)',
 '2 fases Asc / Acc (274.1.5.2)',
 '2 FASES (274.1.5.2)',
 'Caza (239)',
 'A sin cronómetro (238.1.1)',
 'Manga Ganadora (276.2)',
 'C.N. (264)',
 'Dos Mangas (273.3.4)',
 'A con cronómetro (238.2.1)',
 'A c.c. (238.2.1)',
 'Acc y desempate (238.2.2)',
 'A c/c) (273.3.3.1)',
 'Acc (238.1.1)',
 'A c/c) (274.1.5.3)',

In [8]:
# comprobamos cuantos tipos de pruebas unicas tenemos
len(pruebas_unicas)

35

In [9]:
normas = []
for elemento in pruebas_unicas:
    tipo_norma = elemento.split(" ")
    normas.append(tipo_norma[-1])

normas

['(238.2.1)',
 '(238.2.2)',
 '(273.3.3.1)',
 '(274.1.5.3)',
 '(238.2.2)',
 '(238.2.2)',
 '(238.2.2)',
 '(269.3.2)',
 '(238.2.1)',
 '(238.2.1)',
 '(269.3.2)',
 '(269.3.2)',
 '(238.1.2)',
 '(263)',
 '(274.2)',
 '(274.1.5.3)',
 '(274.2)',
 '(238.2.2)',
 '(238.2.1)',
 '(239)',
 '(273.3.3.1)',
 '(274.1.5.2)',
 '(274.1.5.2)',
 '(239)',
 '(238.1.1)',
 '(276.2)',
 '(264)',
 '(273.3.4)',
 '(238.2.1)',
 '(238.2.1)',
 '(238.2.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(274.1.5.3)',
 '(238.2.1)']

In [10]:
len(normas)

35

In [11]:
# volvemos a obtener los valores unicos de los tipos de pruebas
normas_unicas = list(set(normas))
normas_unicas

['(239)',
 '(238.2.1)',
 '(238.2.2)',
 '(269.3.2)',
 '(263)',
 '(274.1.5.2)',
 '(264)',
 '(274.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(276.2)',
 '(238.1.2)',
 '(274.1.5.3)',
 '(273.3.4)']

In [12]:
len(normas_unicas)

14

In [13]:
for prueba in pruebas_unicas:
    norma = normas_unicas[0]
    if isinstance(norma, str) and norma in prueba:
        print(prueba)
    else:
        continue

C (239)
Caza (239)


In [14]:
def creacion_columnas(df, columna_crear, columna, coso = "paises"):
    posiciones = ["RET", "ELI", "NOP"]

    if coso == "paises":
        df[columna_crear] = df[columna].apply(lambda x: x.split("(")[1].strip(")") if x.endswith(")") else "ESP")
        df[columna] = df[columna].apply(lambda x: x.split("(")[0].strip() if x.endswith(")") else x)
    elif coso == "Estado": 
        if "Puntuacion" in df.columns:  
            df["Posicion"] = df.apply(lambda x: "NOP" if pd.isna(x["Puntuacion"]) else x["Posicion"], axis=1)
        df[columna_crear] = df[columna].apply(lambda x: x if x in posiciones else "FIN")
    elif coso == "Premio":
        df[columna_crear] = df[columna].apply(lambda x: False if x == 0 else True)
        
def modificaciones_generales(dataframe):

       dataframe["Prueba"] = dataframe.iloc[3,1]
       dataframe["Fecha_prueba"] = dataframe.iloc[2,1]
       dataframe["Concurso"] = dataframe.iloc[1,1]
       dataframe = dataframe.iloc[9:].reset_index(drop = True)
       dataframe = dataframe.rename(columns = {"Unnamed: 0": "Posicion", 
                                                 "Unnamed: 1": "Lic_jinete",
                                                 "Unnamed: 2": "Jinete", 
                                                 "Unnamed: 3": "Lic_caballo", 
                                                 "Unnamed: 4": "Caballo",
                                                 "Unnamed: 5": "Raza_caballo",
                                                 "Unnamed: 6": "Puntuacion",
                                                 "Unnamed: 7": "Dinero_premio"})
       dataframe["Raza_caballo"] = dataframe["Raza_caballo"].apply(lambda x: None if x == "--" else x)
       creacion_columnas(dataframe, "Pais_jinete", "Jinete")
       creacion_columnas(dataframe, "Pais_caballo", "Caballo")
       creacion_columnas(dataframe, "Estado", "Posicion", coso = "Estado")
       creacion_columnas(dataframe, "Premio", "Dinero_premio", coso = "Premio")
       dataframe = dataframe.reindex(columns = ['Estado','Posicion', 'Lic_jinete', 'Jinete', 'Pais_jinete', 'Lic_caballo', 'Caballo',
              'Raza_caballo', 'Pais_caballo', 'Puntuacion', 'Premio','Dinero_premio', 'Prueba', 'Fecha_prueba',
              'Concurso'])

       return dataframe

In [ ]:
def limpieza_df(df):

    df = modificaciones_generales(df)
    df["Puntuacion"] = df["Puntuacion"].str.split("\n")
    celda = df.iloc[0, -6]

    if len(celda) == 1 and "/" in celda[0]:
        df["Ptos_Obs_R1"] = df["Puntuacion"].apply(lambda x: x[0].split("/")[0])
        df["Ptos_tiempo_R1"] = None
        df["Tiempo_R1"] = None

        df["Ptos_Obs_R2"] = None
        df["Ptos_tiempo_R2"] = None
        df["Tiempo_R2"] = None

        df["Ptos_Obs_R3"] = None
        df["Ptos_tiempo_R3"] = None
        df["Tiempo_R3"] = None
        
    elif len(celda) == 1 and "/" not in celda[0]:
        
        df["Ptos_Obs_R1"] = df["Puntuacion"].apply(lambda x: x[0])
        df["Ptos_tiempo_R1"] = None
        df["Tiempo_R1"] = None

        df["Ptos_Obs_R2"] = None
        df["Ptos_tiempo_R2"] = None
        df["Tiempo_R2"] = None

        df["Ptos_Obs_R3"] = None
        df["Ptos_tiempo_R3"] = None
        df["Tiempo_R3"] = None
    

    # try: 
    #     df[["Puntuacion_1", "Puntuacion_2", "Puntuacion_3"]] = df["Puntuacion"].str.split("\n", expand = True)

    # except ValueError:

    #     try:
    #         df[["Puntuacion_1", "Puntuacion_2"]] = df["Puntuacion"].str.split("\n", expand = True)
    #         df["Puntuacion_3"] = None

    #     except ValueError:

    #         if len 
    #         df["Ptos_Obs_R1"] = df["Puntuacion"].str.split("/")
    #         df["Ptos_Obs_R2"]
    #         df["Ptos_Obs_R3"]
    #         df["Puntuacion_1"] = df["Puntuacion"]
    #         df["Puntuacion_2"] = None
    #         df["Puntuacion_3"] = None

    df = df.drop("Puntuacion", axis = 1)
    return df

In [114]:
coso = "4(4 Obs - 0 Tpo)/70,88"
coso.split("/")

coso

'4(4 Obs - 0 Tpo)/70,88'

In [34]:
normas_unicas

['(239)',
 '(238.2.1)',
 '(238.2.2)',
 '(269.3.2)',
 '(263)',
 '(274.1.5.2)',
 '(264)',
 '(274.2)',
 '(273.3.3.1)',
 '(238.1.1)',
 '(276.2)',
 '(238.1.2)',
 '(274.1.5.3)',
 '(273.3.4)']

In [35]:
# comprobamos los excels que son de tipo Acc con desempate
directorio = "../data/data_salto/resultados_2025"
norma_buscar = normas_unicas[2]
# Obtener la lista de todos los archivos en el directorio
archivos = os.listdir(directorio)

# Filtrar solo los archivos que parecen ser de Excel (puedes refinar esto más si es necesario)
archivos_excel = [f for f in archivos if f.endswith(('.xlsx', '.xls'))]
dfs = []
if archivos_excel:
    # Ordenar la lista de archivos para asegurar que el "primero" sea consistente
    archivos_excel.sort()

    for nombre_excel in archivos_excel:
        ruta_excel = os.path.join(directorio, nombre_excel)
        try:
            # Leer el archivo Excel actual
            df = pd.read_excel(ruta_excel)
            celda = df.iloc[3,1].split(" ")[-1]
            if celda == norma_buscar:
                 print(f"{nombre_excel}")
                 df = limpieza_df(df)
                #  print(df.head(2))
                 dfs.append(df)
            else:
                  continue
        except FileNotFoundError:
            print(f"Error: No se encontró el archivo: {ruta_excel}")
        except Exception as e:
            print(f"Ocurrió un error al leer el archivo {nombre_excel}: {e}")
else:
        print("No se encontraron archivos Excel en el directorio.") 

if dfs:
    df_concat = pd.concat(dfs, ignore_index = True)
    print("df concatenado")
else:
    print("No se encontraron DataFrames para concatenar.")
    
print(norma_buscar)  
df_concat 

RFHE_SALTO_Resultados de la Prueba (12).xls
RFHE_SALTO_Resultados de la Prueba (13).xls
RFHE_SALTO_Resultados de la Prueba (14).xls
RFHE_SALTO_Resultados de la Prueba (23).xls
RFHE_SALTO_Resultados de la Prueba (32).xls
RFHE_SALTO_Resultados de la Prueba (35).xls
RFHE_SALTO_Resultados de la Prueba (5).xls
RFHE_SALTO_Resultados de la Prueba (56).xls
RFHE_SALTO_Resultados de la Prueba (62).xls
RFHE_SALTO_Resultados de la Prueba (7).xls
RFHE_SALTO_Resultados de la Prueba (71).xls
RFHE_SALTO_Resultados de la Prueba (74).xls
RFHE_SALTO_Resultados de la Prueba (79).xls
RFHE_SALTO_Resultados de la Prueba (84).xls
RFHE_SALTO_Resultados de la Prueba (90).xls
RFHE_SALTO_Resultados de la Prueba (93).xls
RFHE_SALTO_Resultados de la Prueba (96).xls
RFHE_SALTO_Resultados de la Prueba - 2025-05-13T125209.767.xls
RFHE_SALTO_Resultados de la Prueba - 2025-05-13T125302.921.xls
RFHE_SALTO_Resultados de la Prueba - 2025-05-13T125338.235.xls
RFHE_SALTO_Resultados de la Prueba - 2025-05-13T125507.968.xls
RF

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Puntuacion_1,Puntuacion_2,Puntuacion_3
0,FIN,1,10204492,Maxime LANTIEZ,FRA,109AJ19,CHACCOLINA VAN'T HUKA Z,None,FRA,True,125,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15,SPRING MET I 2025 YH* (14/01/2025 - 16/01/2025),"0/39,78","0/29,42",None
1,FIN,2,10022941,Jon IDIART,FRA,109AL48,KADANCE DU BIDOU,None,FRA,True,100,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15,SPRING MET I 2025 YH* (14/01/2025 - 16/01/2025),"0/40,06","0/31,25",None
2,FIN,3,10033141,Armand LECOMTE,FRA,108ZS23,KING DES SABLONS,None,FRA,True,75,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15,SPRING MET I 2025 YH* (14/01/2025 - 16/01/2025),"0/41,74","0/31,45",None
3,FIN,4,10042044,Tressy MUHR,FRA,108ZX26,KANNONBALL GRAM'S,None,FRA,True,50,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15,SPRING MET I 2025 YH* (14/01/2025 - 16/01/2025),"0/42,03","0/33,04",None
4,FIN,5,10143006,Leo CITERNE,FRA,109AL39,KLIMT DES BRIMBELLES,None,FRA,True,35,"5 años - 1,15 - A.c.c. y desempate (238.2.2)",2025-01-15,SPRING MET I 2025 YH* (14/01/2025 - 16/01/2025),"0/47,07","0/34,07",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9140,RET,RET,333375,Charlotte FIEVET .,ESP,155410,CARIOCA,OLDBG,ESP,False,0,"1,40 gp - A.c.c. y desempate al cronómetro (23...",2025-04-27,CSN3* Copa Presidente (25/04/2025 - 27/04/2025),RET,None,None
9141,RET,RET,15738,Emilio PERALTA CAMPAÑA,ESP,146657,LE DOUGLAS VAN DE PEELVENHOEVE,KWPN,ESP,False,0,"1,40 gp - A.c.c. y desempate al cronómetro (23...",2025-04-27,CSN3* Copa Presidente (25/04/2025 - 27/04/2025),RET,None,None
9142,ELI,ELI,15696,Mickey SEAQUIST FERNANDEZ-CAVADA,ESP,154371,CONTADOR VD DORPERMOLEN Z,ZANG,ESP,False,0,"1,40 gp - A.c.c. y desempate al cronómetro (23...",2025-04-27,CSN3* Copa Presidente (25/04/2025 - 27/04/2025),ELI,None,None
9143,NOP,NOP,3673,Jorge BIDÓN DAURELLA,ESP,141858,CORMIRA BY P&S Z,ZANG,ESP,False,0,"1,40 gp - A.c.c. y desempate al cronómetro (23...",2025-04-27,CSN3* Copa Presidente (25/04/2025 - 27/04/2025),NOP,None,None


## Pruebas norma (264): Copa de naciones

Son a dos rondas siempre, a excepción de que haya empate entre los primeros equipos, caso en el que habrá un desempate, llevando así a haber tres rondas

In [34]:
cosita = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T173909.958.xls")
cosita = modificaciones_generales(cosita)
try:
    cosita[["Puntuacion_1", "Puntuacion_2", "Puntuacion_3"]] = cosita["Puntuacion"].str.split("\n", expand = True)

except ValueError:
    cosita[["Puntuacion_1", "Puntuacion_2"]] = cosita["Puntuacion"].str.split("\n", expand = True)
    cosita["Puntuacion_3"] = None
cosita = cosita.drop("Puntuacion", axis = 1)
cosita.head(5)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Puntuacion_1,Puntuacion_2,Puntuacion_3
0,FIN,1,10188567,Carlos BORHO GIRON,ESP,107JQ37,DAMASKUS 57,HANN,ESP,False,0,J - CN - C.N. (264),2025-03-14,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,"4/74,63","12/72,23","0/36,23"
1,FIN,1,10209103,Borja GARCÍA SÁNCHEZ,ESP,106EM83,CRONOS D´ELLE,SF,ESP,False,0,J - CN - C.N. (264),2025-03-14,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,"4/71,87","8/71,65","4/36,96"
2,FIN,1,10296116,Haizea GARMENDIA AJURIA,ESP,105MZ90,CONTHARGOS ROUGE,OLDBG,ESP,False,0,J - CN - C.N. (264),2025-03-14,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,"0/71,75","4/68,81","4/36,28"
3,FIN,1,10209247,Celia COBO TERAN,ESP,105OI46,ALTO DU ROC,SF,ESP,False,0,J - CN - C.N. (264),2025-03-14,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,"4/67,66","4/70,91","0/36,38"
4,FIN,2,10283797,Martim RIBEIRO,POR,105HR21,FELIBERLINA T,None,POR,False,0,J - CN - C.N. (264),2025-03-14,Andalucía Sunshine Tour CSIOYJCh 2025 (13/03/2...,"0/70,42","4/68,63","0/35,01"


## Pruebas norma (276.2): Una Manga con Manga Ganadora

In [82]:
manga_ganadora = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T191013.946.xls")
manga_ganadora = modificaciones_generales(manga_ganadora)
# manga_ganadora[["Puntuacion_1", "Puntuacion_2"]] = manga_ganadora["Puntuacion"].str.split("\n", expand = True)
# manga_ganadora["Puntuacion_3"] = None
# manga_ganadora = manga_ganadora.drop("Puntuacion", axis = 1)
manga_ganadora["Puntuacion"] = manga_ganadora["Puntuacion"].str.split("\n")
print(len(manga_ganadora.iloc[0,-1]))
# for elemento in manga_ganadora["Puntuacion"]:
#     print(len(elemento))
manga_ganadora

75


,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso
0,FIN,1,264438,Izaro JUNQUERA LÓPEZ,ESP,157673,DANAH,CRUZ,ESP,"[0/67,62, 0/36,00]",True,600,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
1,FIN,2,242355,Haizea GARMENDIA AJURIA,ESP,120669,CONTHARGOS ROUGE,OLDBG,ESP,"[0/68,15, 0/36,61]",True,400,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
2,FIN,3,192930,Aitana REGIL CUÑARRO,ESP,146510,QUITO SC,CDE,ESP,"[0/69,26, 0/38,29]",True,300,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
3,FIN,4,229720,Paula REGIL CUÑARRO,ESP,159183,ERALD SAUGETS,SF,ESP,"[0/67,05, 0/39,85]",True,200,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
4,FIN,5,3746,Xabier ZABALETA SUSPERREGUI,ESP,156724,NORWAY VZ,KWPN,ESP,"[0/72,09, 0/40,54]",True,150,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
5,FIN,6,5147,Pablo GONZÁLEZ PEDROSA,ESP,139436,ARISA DEL POMAR,CDE,ESP,"[0/71,81, 0/42,57]",True,125,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
6,FIN,7,219745,Irene ZUAZUA DE LA TORRE,ESP,127959,KIMONO DE DUERO,CDE,ESP,"[0/69,79, 4(4 Obs - 0 Tpo)/37,65]",True,100,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
7,FIN,8,2360,Pello ELORDUY IBARZABAL,ESP,149570,HERO DU PRE,SF,ESP,"[0/67,70, 4(4 Obs - 0 Tpo)/38,49]",True,80,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
8,FIN,9,228514,Jairo JUNQUERA PERERA,ESP,138854,GUY DE MAEZA,CDE,ESP,"[0/68,26, 4(4 Obs - 0 Tpo)/39,69]",True,70,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...
9,FIN,10,2360,Pello ELORDUY IBARZABAL,ESP,140548,FASTOUREL D'OR,SF,ESP,"[0/70,19, 4(4 Obs - 0 Tpo)/43,32]",True,50,"1,30 - Manga Ganadora (276.2)",2025-04-12,CSN*** Spring Showjumping - Heras Horses & Eve...


## Pruebas norma (238.1.1): Baremo A, sin cronómetro
Los igualados a puntos comparten premio y según las condiciones del programa se pueden estipular uno o dos desempates sin cronómetro para los igualados a puntos para el primer puesto.

In [127]:
BA_sc = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T151603.419.xls")
BA_sc = modificaciones_generales(BA_sc)
# BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].str.split("/")
# BA_sc["Puntuacion_1"] = BA_sc["Puntuacion"]
# BA_sc["Puntuacion_2"] = None
# BA_sc["Puntuacion_3"] = None
# BA_sc = BA_sc.drop("Puntuacion", axis = 1)
#  coso = BA_sc.iloc[3,9]
# print(coso.split("/")[0].split("(")[1][0])
# print(coso.split("/")[0].split("(")[1][-6])
# print(coso.split("/")[1])

BA_sc["Puntuacion"] = BA_sc["Puntuacion"].str.split("\n")

# for elemento in BA_sc["Puntuacion"]:
#     print(len(elemento))

celda = BA_sc.iloc[0, -6]
if len(celda) == 1:

    if "/" in celda[0]:
        celda_1 = celda[0].split("/")

        if "(" not in celda_1:
            BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0])
            BA_sc["Ptos_tiempo_R1"] = None
            BA_sc["Tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[1])

            BA_sc["Ptos_Obs_R2"] = None
            BA_sc["Ptos_tiempo_R2"] = None
            BA_sc["Tiempo_R2"] = None

            BA_sc["Ptos_Obs_R3"] = None
            BA_sc["Ptos_tiempo_R3"] = None
            BA_sc["Tiempo_R3"] = None

        elif "(" in celda_1:
            BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0].split("(")[1][0])
            BA_sc["Ptos_tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[0].split("(")[1][-6])
            BA_sc["Tiempo_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0].split("/")[1])

            BA_sc["Ptos_Obs_R2"] = None
            BA_sc["Ptos_tiempo_R2"] = None
            BA_sc["Tiempo_R2"] = None

            BA_sc["Ptos_Obs_R3"] = None
            BA_sc["Ptos_tiempo_R3"] = None
            BA_sc["Tiempo_R3"] = None
        else:
            pass

    elif "/" not in celda[0]:

        BA_sc["Ptos_Obs_R1"] = BA_sc["Puntuacion"].apply(lambda x: x[0])
        BA_sc["Ptos_tiempo_R1"] = None
        BA_sc["Tiempo_R1"] = None

        BA_sc["Ptos_Obs_R2"] = None
        BA_sc["Ptos_tiempo_R2"] = None
        BA_sc["Tiempo_R2"] = None

        BA_sc["Ptos_Obs_R3"] = None
        BA_sc["Ptos_tiempo_R3"] = None
        BA_sc["Tiempo_R3"] = None
BA_sc
# BA_sc.iloc[3,9][0].split("/")[0].split("(")[1][-6]

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,10040364,Luis FRANCO MARTÍNEZ,ESP,109DA71,FENIX ANANTARA,CDE,ESP,"[0/74,05]",True,75,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"74,05",None,None,None,None,None,None
1,FIN,2,10116527,Giorgia DE LORENZI,ITA,109CB95,BADDALOU,None,ITA,"[0/76,34]",True,60,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"76,34",None,None,None,None,None,None
2,FIN,3,10005772,Kevin GIELEN,BEL,109BF73,CHARLY VAN HET CAUTERHOF Z,None,BEL,"[0/77,69]",True,45,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,0,None,"77,69",None,None,None,None,None,None
3,FIN,4,10289081,Christina Elisabeth Norgaard RUD,NOR,109AW75,UNICO DOMM,None,NOR,"[4(4 Obs - 0 Tpo)/70,88]",True,30,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4(4 Obs - 0 Tpo),None,"70,88",None,None,None,None,None,None
4,FIN,5,10104173,Valeria BIAZZI,ITA,109CC29,BRIGITTE BELLE,None,ITA,"[4(4 Obs - 0 Tpo)/73,38]",True,21,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4(4 Obs - 0 Tpo),None,"73,38",None,None,None,None,None,None
5,FIN,6,10139698,Ronnie JONES,GBR,109BS53,SLH ICAN DOMINATE,None,GBR,"[4(4 Obs - 0 Tpo)/75,42]",True,16.5,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4(4 Obs - 0 Tpo),None,"75,42",None,None,None,None,None,None
6,FIN,7,10025853,Gilles BOTTON,BEL,109AN17,COMMILF'OH DE LA MARCHETTE Z,None,BEL,"[4(4 Obs - 0 Tpo)/75,48]",True,12,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,4(4 Obs - 0 Tpo),None,"75,48",None,None,None,None,None,None
7,FIN,8,10138266,Scott DOLLEMORE,GBR,109CJ54,BEA BLUE PS,None,GBR,"[8(8 Obs - 0 Tpo)/73,56]",True,9,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8(8 Obs - 0 Tpo),None,"73,56",None,None,None,None,None,None
8,FIN,9,10114636,Fidel DÁVILA GARCÍA,ESP,109CI39,KAN DE LLAMOSAS,SF,ESP,"[8(8 Obs - 0 Tpo)/74,43]",True,9,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8(8 Obs - 0 Tpo),None,"74,43",None,None,None,None,None,None
9,FIN,10,10048768,Emilie PAILLOT,SUI,109BY27,PRADA ES,None,SUI,"[8(8 Obs - 0 Tpo)/79,54]",True,7.5,"5 años - 1,15 - A sin cronómetro (238.1.1)",2025-02-19,CSIYH* - 3rd Week XXXI Andalucía Sunshine Tour...,8(8 Obs - 0 Tpo),None,"79,54",None,None,None,None,None,None


## Pruebas norma (239): Baremo C
Las faltas son penalizadas en segundos que se agregan al tiempo empleado por el aprticipante en completar el recorrido o eliminación. El resultado se determinará como la suma del tiempo del recorrido incluidas las correcciones de tiempo, si las hubiera, más cuatro segundos por obstáculo derribado (tres segundos en desempates según baremo C o en la segunda fase de la prueba de dos fases o en eliminatorias sucesivas) da el resultado en segundos, para el participante.
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T192941.693.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T193001.145.xls

In [95]:
pd.set_option('display.max_columns', None)


In [107]:
BC = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T193001.145.xls")
BC = modificaciones_generales(BC)
# BC["Puntuacion_1"] = BC["Puntuacion"].str.split("\n", expand = True)
# BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x.split("/")[0])
BC["Puntuacion"] = BC["Puntuacion"].str.split("\n")

if len(BC.iloc[0, -6]) == 1 and "/" in BC.iloc[0, -6][0]:
    BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x[0].split("/")[0])
    BC["Ptos_tiempo_R1"] = None
    BC["Tiempo_R1"] = None

    BC["Ptos_Obs_R2"] = None
    BC["Ptos_tiempo_R2"] = None
    BC["Tiempo_R2"] = None

    BC["Ptos_Obs_R3"] = None
    BC["Ptos_tiempo_R3"] = None
    BC["Tiempo_R3"] = None
    
elif len(BC.iloc[0, -6]) == 1 and "/" not in BC.iloc[0, -6][0]:
    
    BC["Ptos_Obs_R1"] = BC["Puntuacion"].apply(lambda x: x[0])
    BC["Ptos_tiempo_R1"] = None
    BC["Tiempo_R1"] = None

    BC["Ptos_Obs_R2"] = None
    BC["Ptos_tiempo_R2"] = None
    BC["Tiempo_R2"] = None

    BC["Ptos_Obs_R3"] = None
    BC["Ptos_tiempo_R3"] = None
    BC["Tiempo_R3"] = None
# len(BC.iloc[0,-1])
# BC["Puntuacion_2"] = None
# BC["Puntuacion_3"] = None
# BC = BC.drop("Puntuacion", axis = 1)

BC

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Ptos_Obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_Obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_Obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,45827,Luna JIMÉNEZ BECHOLD,ESP,165666,SUPREME DU MILON MERCADO DEL AGRICULTOR,SF,ESP,"[79,10]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"79,10",None,None,None,None,None,None,None,None
1,FIN,2,70254,Alejandra MELIAN SANTANA,ESP,167713,IL FOTOGRAFO,PSI,ESP,"[80,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"80,58",None,None,None,None,None,None,None,None
2,FIN,3,45827,Luna JIMÉNEZ BECHOLD,ESP,162922,OHIO LV VAN T BIEZENHOF,BWP,ESP,"[92,42]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"92,42",None,None,None,None,None,None,None,None
3,FIN,4,155208,Loic MOLINA MÁRQUEZ,ESP,155995,GWYNETH,KWPN,ESP,"[96,75]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"96,75",None,None,None,None,None,None,None,None
4,FIN,5,273709,Saul Alejandro SÁNCHEZ ROJAS,ESP,168850,GIMORA,KWPN,ESP,"[101,53]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"101,53",None,None,None,None,None,None,None,None
5,FIN,6,38323,Nicole STAIGER,ESP,146915,BENALUP C,CDE,ESP,"[105,58]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"105,58",None,None,None,None,None,None,None,None
6,FIN,7,33381,Isabel PALLERO DÁVILA,ESP,90270,MARISCAL DEL PAS,CDE,ESP,"[115,45]",False,0,"1,30 - Caza (239)",2025-04-19,CSN* C.H. DEL SUR - SEMANA SANTA (17/04/2025 -...,"115,45",None,None,None,None,None,None,None,None


## Pruebas norma (273.3.3.1): Prueba en dos Mangas

Ambas mangas en Baremo A, ambas con cronómetro.

- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T155509.489.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T163100.768.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T185621.773.xls
- RFHE_SALTO_Resultados de la Prueba - 2025-05-13T192327.719.xls

In [62]:
Dos_m = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T155509.489.xls")
Dos_m = modificaciones_generales(Dos_m)
Dos_m[["Puntuacion_1", "Puntuacion_2"]] = Dos_m["Puntuacion"].str.split("\n", expand = True)
Dos_m["Puntuacion_3"] = None
Dos_m = Dos_m.drop("Puntuacion", axis = 1)
Dos_m.head(8)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso,Puntuacion_1,Puntuacion_2,Puntuacion_3
0,FIN,1,5042,Pedro Fernando MATEOS RODRÍGUEZ,ESP,149266,QUIZAS CC,CDE,ESP,True,1000,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"0/67,01","0/42,65",None
1,FIN,2,274244,Alex CODINA LERIA,ESP,155917,COSTELLO DC,CDE,ESP,True,800,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"0/69,79","0/46,09",None
2,FIN,3,132635,Pilar MATEOS DE URBINA,ESP,147592,AZURE DES ROCHES,SF,ESP,True,600,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"0/71,58","0/47,71",None
3,FIN,4,369989,Emmy Pauline KROGER KROGER,ESP,170700,DSP DACENTO H,REITF,ESP,True,400,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"0/70,10","0/47,74",None
4,FIN,5,323398,Alex TEJADA BEROIZ,ESP,116440,APOLO DE EQUIANPE,CDE,ESP,True,300,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"4(4 Obs - 0 Tpo)/66,87","0/44,20",None
5,FIN,6,15975,Alvaro SANCHEZ-ARCILLA ARTIÑANO,ESP,156003,CHAMP BLUE,REITF,ESP,True,200,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"0/69,20","4(4 Obs - 0 Tpo)/44,83",None
6,FIN,7,5018,Javier José MATEOS RODRÍGUEZ,ESP,143177,LEVIS C,HANN,ESP,True,150,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"4(4 Obs - 0 Tpo)/67,53","0/47,08",None
7,FIN,8,14425,Carmen GARCIA-TORRES SOLANO,ESP,159657,FOR JEWEL EDM Z,BWP,ESP,True,140,"1,40 gp - Dos Mangas sin desempate (273.3.3.1)",2025-02-23,CSN3* F.H.M. - En Club Las Cadenas (22/02/20...,"4(4 Obs - 0 Tpo)/67,18","0/48,14",None


## Pruebas norma (273.3.4): Prueba en dos Mangas

Ambas mangas en Baremo A, ambas con cronómetro.

In [67]:
Dos_m = pd.read_excel("../data/data_salto/resultados_2025/RFHE_SALTO_Resultados de la Prueba - 2025-05-13T165920.069.xls")
Dos_m = modificaciones_generales(Dos_m)
# Dos_m[["Puntuacion_1", "Puntuacion_2"]] = Dos_m["Puntuacion"].str.split("\n", expand = True)
# Dos_m["Puntuacion_3"] = None
# Dos_m = Dos_m.drop("Puntuacion", axis = 1)
Dos_m.head(8)

,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,Premio,Dinero_premio,Prueba,Fecha_prueba,Concurso
0,FIN,1,10016473,Penelope LEPREVOST,FRA,106VL82,DJAGGER SEMILLY,None,FRA,"0/59,15",True,7050,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
1,FIN,2,10150533,Finn BOEREKAMP,NED,106RM98,JAY JAY ES,KWPN,ESP,"0/59,68",True,5640,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
2,FIN,3,10045115,Anthony BOURQUARD,SUI,108CC64,FLANAGAN SEMILLY,None,SUI,"0/60,28",True,4230,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
3,FIN,4,10058729,Rodrigo Giesteira ALMEIDA,POR,107DF31,COMME LE COEUR,None,GER,"0/60,49",True,2820,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
4,FIN,5,10097112,Sanne THIJSSEN,NED,107FW98,LOLANDRIA Z,None,NED,"0/61,21",True,1974,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
5,FIN,6,10002057,Luca Maria MONETA,ITA,108CQ45,EL CAPONE RK,None,FRA,"0/62,35",True,1551,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
6,FIN,7,10017385,William WHITAKER,GBR,107HD18,MILLFIELD QUICKSTEP,None,GBR,"0/64,63",True,1128,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...
7,FIN,8,10098599,William FLETCHER,GBR,108PH34,CALIXTE HEARTBREAKER Z,None,GBR,"0/64,75",True,846,"1,45 (GP) - Dos Mangas (273.3.4)",2025-03-08,CSI4* - 5th Week XXXI Andalucía Sunshine Tour ...


# Valores jsons

In [3]:
import json

# Cargar el JSON
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

# Obtener la longitud de referencia
longitud_maxima = max(len(v) for v in datos_concursos.values() if isinstance(v, list))

# Listas a revisar
listas_objetivo = ["Inicio", "Final", "Ambito"]

# Detectar y mostrar valores faltantes
for key in listas_objetivo:
    lista = datos_concursos[key]
    indices_presentes = set(range(len(lista)))
    indices_totales = set(range(longitud_maxima))
    indices_faltantes = sorted(indices_totales - indices_presentes)

    print(f"\n🔎 Valores faltantes en la lista '{key}':")

    for idx in indices_faltantes:
        nombre = datos_concursos["Nombre"][idx] if idx < len(datos_concursos["Nombre"]) else "(sin nombre)"
        fecha_inicio = datos_concursos["Inicio"][idx] if idx < len(datos_concursos["Inicio"]) else "(sin fecha)"
        fecha_final = datos_concursos["Final"][idx] if idx < len(datos_concursos["Final"]) else "(sin fecha)"
        print(f"- Índice {idx} → concurso: {nombre} | Inicio: {fecha_inicio} | Final: {fecha_final}")




🔎 Valores faltantes en la lista 'Inicio':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Final':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)

🔎 Valores faltantes en la lista 'Ambito':
- Índice 148 → concurso: CSI1* Opglabbeek | Inicio: (sin fecha) | Final: (sin fecha)


In [4]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


In [7]:
import json

# Cargar datos de pruebas
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Longitud total según el campo más largo
longitud_total = len(datos_pruebas["Concurso"])

# Campos a verificar
campos_objetivo = ["Prueba", "Categoría", "Fecha"]  # puedes añadir más si quieres

# Verificar faltantes y mostrar con fecha si existe
for campo in campos_objetivo:
    if campo not in datos_pruebas:
        continue
    
    lista = datos_pruebas[campo]
    indices_faltantes = set(range(longitud_total)) - set(range(len(lista)))
    
    if indices_faltantes:
        print(f"\n🔍 Faltantes en '{campo}':")
        for i in sorted(indices_faltantes):
            fecha = datos_pruebas["Fecha"][i] if i < len(datos_pruebas["Fecha"]) else "(sin fecha)"
            concurso = datos_pruebas["Concurso"][i] if i < len(datos_pruebas["Concurso"]) else "(sin concurso)"
            print(f"- Índice {i} → concurso: '{concurso}' | fecha: {fecha}")





🔍 Faltantes en 'Prueba':
- Índice 14957 → concurso: 'CSI2* LIER (BEL)' | fecha: 115-125 - Two phases, both atc T A
- Índice 14958 → concurso: 'CSI2* LIER (BEL)' | fecha: CSI YH*
- Índice 14959 → concurso: 'CSI2* LIER (BEL)' | fecha: Disponibles
- Índice 14960 → concurso: 'CSI2* LIER (BEL)' | fecha: Salto de Obstáculos
- Índice 14961 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14962 → concurso: 'CSI2* LIER (BEL)' | fecha: 16/12/2020
- Índice 14963 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14964 → concurso: 'CSI2* LIER (BEL)' | fecha: 17/12/2020
- Índice 14965 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14966 → concurso: 'CSIYH LIER (BEL)' | fecha: 18/12/2020
- Índice 14967 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14968 → concurso: 'CSIYH LIER (BEL)' | fecha: 19/12/2020
- Índice 14969 → concurso: 'CSIYH LIER (BEL)' | fecha: 20/12/2020
- Índice 14970 → concurso: 'CSI2* LIER (BEL)' | fecha: (sin fecha)
- Índice 14971 → c

In [9]:
with open('../data/data_salto/concursos/concursos_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 149
La longitud de Categoría es: 149
La longitud de Provincia es: 149
La longitud de Localidad es: 149
La longitud de Disciplina es: 149
La longitud de Federación es: 149
La longitud de Resultados es: 149
La longitud de País es: 149
La longitud de Inicio es: 148
La longitud de Final es: 148
La longitud de Ambito es: 148


In [10]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2025.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 1144
La longitud de Fecha es: 1144
La longitud de Prueba es: 1144
La longitud de Categoría es: 1142
La longitud de Número es: 1141
La longitud de Concurso es: 1144


# Completo

In [38]:
with open('../data/data_completo/pruebas/pruebas_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 561
La longitud de Fecha es: 561
La longitud de Prueba es: 561
La longitud de Categoría es: 561
La longitud de Número es: 561
La longitud de Concurso es: 561


In [39]:
with open('../data/data_completo/pruebas/pruebas_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 496
La longitud de Fecha es: 496
La longitud de Prueba es: 496
La longitud de Categoría es: 496
La longitud de Número es: 496
La longitud de Concurso es: 496


In [40]:
with open('../data/data_completo/concursos/concursos_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 480
La longitud de Categoría es: 480
La longitud de Provincia es: 480
La longitud de Localidad es: 480
La longitud de Disciplina es: 480
La longitud de Federación es: 480
La longitud de Resultados es: 480
La longitud de País es: 480
La longitud de Inicio es: 477
La longitud de Final es: 477
La longitud de Ambito es: 477


In [41]:
with open('../data/data_completo/concursos/concursos_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 223
La longitud de Categoría es: 223
La longitud de Provincia es: 223
La longitud de Localidad es: 223
La longitud de Disciplina es: 223
La longitud de Federación es: 223
La longitud de Resultados es: 223
La longitud de País es: 223
La longitud de Inicio es: 223
La longitud de Final es: 223
La longitud de Ambito es: 223
